In [1]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import logging
import tensorflow as tf

# tf.get_logger().setLevel(logging.ERROR)

# Allow memory growth for the GPU
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# Activate mixed precision
from keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")
from dino import create_dino_model, Sequential, layers, ResNet, ResnetEnum

2023-11-28 20:45:18.181930: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 20:45:18.860642: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070 Ti, compute capability 8.6
Using TensorFlow backend


2023-11-28 20:45:19.899235: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-28 20:45:19.923246: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-28 20:45:19.923305: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-28 20:45:19.923757: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
BATCH_SIZE = 2

In [4]:
# All images under this path will be used for training
dataset_path = "/mnt/c/Users/mnusr/PycharmProjects/hcc_tests/big_image_example/processed_2"

# All images under this path will be used for testing
test_dataset_path = "/mnt/c/Users/mnusr/PycharmProjects/hcc_tests/datasets/hcc_vs_periph"

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    label_mode=None,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(2048, 2048),
    shuffle=True,
    seed=42,
    interpolation="bicubic",
).prefetch(tf.data.AUTOTUNE).take(32 * 1)

Found 38992 files belonging to 1 classes.


In [4]:
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dataset_path,
    label_mode='binary',
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(512, 512),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training",
    interpolation="bicubic",
).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dataset_path,
    label_mode='binary',
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(512, 512),
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation",
    interpolation="bicubic",
).prefetch(tf.data.AUTOTUNE)

Found 2809 files belonging to 2 classes.
Using 2248 files for training.
Found 2809 files belonging to 2 classes.
Using 561 files for validation.


In [5]:
# Zero label count
total_zero_labels = 0
total_one_labels = 0
for batch in validation_dataset:
    total_zero_labels += tf.math.count_nonzero(batch[1] == 0)
    total_one_labels += tf.math.count_nonzero(batch[1] == 1)

print(f"Total zero labels: {total_zero_labels}")
print(f"Total one labels: {total_one_labels}")
print(f"Total labels: {total_zero_labels + total_one_labels}")
class_weight = {0: total_one_labels / (total_zero_labels + total_one_labels),
                1: total_zero_labels / (total_zero_labels + total_one_labels)}
print(f"Class weight: {class_weight}")

Total zero labels: 1339
Total one labels: 909
Total labels: 2248
Class weight: {0: <tf.Tensor: shape=(), dtype=float64, numpy=0.4043594306049822>, 1: <tf.Tensor: shape=(), dtype=float64, numpy=0.5956405693950177>}


In [6]:
# Zero label count
total_zero_labels = 0
total_one_labels = 0
for batch in test_dataset:
    total_zero_labels += tf.math.count_nonzero(batch[1] == 0)
    total_one_labels += tf.math.count_nonzero(batch[1] == 1)

print(f"Total zero labels: {total_zero_labels}")
print(f"Total one labels: {total_one_labels}")
print(f"Total labels: {total_zero_labels + total_one_labels}")
class_weight_test = {0: total_one_labels / (total_zero_labels + total_one_labels),
                     1: total_zero_labels / (total_zero_labels + total_one_labels)}
print(f"Class weight: {class_weight_test}")

Total zero labels: 331
Total one labels: 230
Total labels: 561
Class weight: {0: <tf.Tensor: shape=(), dtype=float64, numpy=0.40998217468805703>, 1: <tf.Tensor: shape=(), dtype=float64, numpy=0.5900178253119429>}


In [3]:
teacher = ResNet(
    rescale=True,
    input_shape=(None, None, 3),
    batch_count=BATCH_SIZE,
    activations="gelu",
    resnet_version=ResnetEnum.resnet16,
)

student = ResNet(
    rescale=True,
    input_shape=(None, None, 3),
    batch_count=BATCH_SIZE,
    activations="gelu",
    resnet_version=ResnetEnum.resnet16,
)

model = create_dino_model(
    student,
    teacher,
    global_crop_size=512,
    local_crop_size=256,
    local_augmentation_count=4,
    network_momentum=0.99995,
    teacher_temperature=0.01
)

schedule = tf.keras.optimizers.schedules.CosineDecay(0.00001, decay_steps=152 * 80, alpha=1e-6, warmup_target=0.00009,
                                                     warmup_steps=152 * 150)

optimizer = tf.keras.optimizers.Adam(schedule,weight_decay=0.04)#,clipnorm=3)

scaled_optimizer = mixed_precision.LossScaleOptimizer(optimizer)

model.compile(optimizer=scaled_optimizer, run_eagerly=False, jit_compile=False)



2023-11-28 20:45:26.140004: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-28 20:45:26.140109: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-28 20:45:26.140151: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-28 20:45:26.308201: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-28 20:45:26.308273: I tensorflow/compile

In [5]:
student.trainable = True
model.fit(
    train_dataset,
    epochs=500
)

Epoch 1/500
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow
INFO:tensorflow:Running stacked flow


ValueError: in user code:

    File "/home/nusret/miniconda3/envs/tf/lib/python3.11/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/nusret/miniconda3/envs/tf/lib/python3.11/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/nusret/miniconda3/envs/tf/lib/python3.11/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/mnt/c/Users/mnusr/PycharmProjects/tf_model_implementations/tf_model_implementations/cv/self_supervised/dino.py", line 268, in train_step
        tf.cond(
    File "/mnt/c/Users/mnusr/PycharmProjects/tf_model_implementations/tf_model_implementations/cv/self_supervised/dino.py", line 281, in apply_accumulated_gradients
        variable.assign(tf.math.divide(variable, tf.cast(self.accumulation_steps, tf.flo

In [ ]:
student.trainable = False

model_st = Sequential(
    [
        student,
        layers.Dense(1),
        layers.Activation("sigmoid", dtype=tf.float32)
    ])

optimizer_st = tf.keras.optimizers.Adam(0.00001)

scaled_optimizer_st = mixed_precision.LossScaleOptimizer(optimizer_st)

model_st.compile(optimizer=scaled_optimizer_st, run_eagerly=False, jit_compile=True,
                 metrics=[
                     tf.metrics.BinaryAccuracy(name='accuracy'),
                     tf.metrics.AUC(name='auc'),
                     tf.metrics.Precision(name='precision'),
                     tf.metrics.Recall(name='recall'),
                 ],
                 loss="binary_crossentropy")

In [ ]:
model_st.fit(
    validation_dataset,
    validation_data=test_dataset,
    epochs=20,
    initial_epoch=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
    ,
    class_weight=class_weight
)

In [ ]:
#model.save('dino')
model_st.save('dino_student')

In [ ]:
# Try with imagenet pretrained resnet50
model_pt = tf.keras.applications.resnet_rs.ResNetRS50(pooling='avg',input_shape=[512,512,3],include_top=False)
model_pt.trainable = False
model_pt = Sequential(
    [
        model_pt,
        layers.Dense(1),
        layers.Activation("sigmoid", dtype=tf.float32)
    ]
)

optimizer_pt = tf.keras.optimizers.Adam(0.00001)

scaled_optimizer_pt = mixed_precision.LossScaleOptimizer(optimizer_pt)

model_pt.compile(optimizer=scaled_optimizer_pt, run_eagerly=False, jit_compile=True,
                 metrics=[
                     tf.metrics.BinaryAccuracy(name='accuracy'),
                     tf.metrics.AUC(name='auc'),
                     tf.metrics.Precision(name='precision'),
                     tf.metrics.Recall(name='recall'),
                 ],
                 loss="binary_crossentropy")

In [ ]:
model_pt.fit(
    validation_dataset,
    validation_data=test_dataset,
    epochs=20,
    initial_epoch=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
    ,
    class_weight=class_weight
)

In [ ]:
model_pt.predict(test_dataset)

In [ ]:
model_pt.summary()

In [ ]:
model_st.summary()